In [1]:
experimentsPath = "/raid/elhamod/CIFAR_HGNN/experiments/"
dataPath = "/raid/elhamod/"
experimentName = "CIFAR_phylogeny_8_4_0_KLDiv"
device = 0
detailed_reporting = True

In [ ]:

experimetnsFileName = "experiments.csv"

In [2]:
import matplotlib.pyplot as plt
import torch
import sys
import os
from sklearn.metrics import f1_score
import pandas as pd
from tqdm import tqdm
from tqdm.auto import trange

from myhelpers import config_plots, TrialStatistics
from HGNN.train import CNN, dataLoader
from myhelpers import cifar_dataLoader
from HGNN.train.configParser import ConfigParser, getModelName, getDatasetName
config_plots.global_settings()


experimentPathAndName = os.path.join(experimentsPath, experimentName)
# set cuda
if device is not None:
    print("using cuda", device)
    torch.cuda.set_device(device)

else:
    print("using cpu")

# get experiment params
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)

# init experiments file
experimentsFileNameAndPath = os.path.join(experimentsPath, experimetnsFileName)

paramsIterator = config_parser.getExperiments()  
number_of_experiments = sum(1 for e in paramsIterator)  
experiment_index = 0

# Loop through experiments
# with progressbar.ProgressBar(max_value=number_of_experiments) as bar:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for experiment_params in config_parser.getExperiments():
        print(experiment_params)
        experimentHash =TrialStatistics.getTrialName(experiment_params)

        # load images 
        datasetManager = dataLoader.datasetManager(experimentPathAndName, dataPath)
        datasetManager.updateParams(config_parser.fixPaths(experiment_params))
        train_loader, validation_loader, test_loader = datasetManager.getLoaders()
        architecture = {
            "fine": len(train_loader.dataset.csv_processor.getFineList()),
            "coarse" : len(train_loader.dataset.csv_processor.getCoarseList())
        }

        # Loop through n trials
        for i in trange(experiment_params["numOfTrials"], desc="trial"):
            modelName = getModelName(experiment_params, i)
            trialName = os.path.join(experimentPathAndName, modelName)
            trialHash = TrialStatistics.getTrialName(experiment_params, i)

            row_information = {
                'experimentName': experimentName,
                'modelName': modelName,
                'datasetName': getDatasetName(config_parser.fixPaths(experiment_params)),
                'experimentHash': experimentHash,
                'trialHash': trialHash
            }
            row_information = {**row_information, **experiment_params} 
            print(row_information)

            # Train/Load model
            model = CNN.create_model(architecture, experiment_params, device=device)

            if os.path.exists(CNN.getModelFile(trialName)):
                print("Model {0} found!".format(trialName))
            else:
                initModelPath = CNN.getInitModelFile(experimentPathAndName)
                if os.path.exists(initModelPath):
                    model.load_state_dict(torch.load(initModelPath))
                    print("Init Model {0} found!".format(initModelPath))
                CNN.trainModel(train_loader, validation_loader, experiment_params, model, trialName, test_loader, device=device, detailed_reporting=detailed_reporting)

            # Add to experiments file
            if os.path.exists(experimentsFileNameAndPath):
                experiments_df = pd.read_csv(experimentsFileNameAndPath)
            else:
                experiments_df = pd.DataFrame()

            record_exists = not (experiments_df[experiments_df['modelName'] == modelName][experiments_df['experimentName'] == experimentName]).empty if not experiments_df.empty else False
            if record_exists:
                experiments_df.drop(experiments_df[experiments_df['modelName'] == modelName][experiments_df['experimentName'] == experimentName].index, inplace = True) 

            experiments_df = experiments_df.append(pd.DataFrame(row_information, index=[0]), ignore_index = True)
            experiments_df.to_csv(experimentsFileNameAndPath, header=True, index=False)

        bar.update()

        experiment_index = experiment_index + 1



Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: mndhamod (use `wandb login --relogin` to force relogin)
experiment:   0%|          | 0/1 [00:00<?, ?it/s]

using cuda 0
{'image_path': 'cifar-100-python', 'suffix': None, 'img_res': 32, 'augmented': False, 'batchSize': 128, 'learning_rate': 5e-05, 'numOfTrials': 5, 'fc_layers': 1, 'modelType': 'BB', 'lambda': 0.01, 'unsupervisedOnTest': False, 'tl_model': 'CIFAR', 'link_layer': 'avgpool', 'adaptive_smoothing': False, 'adaptive_lambda': 0.015, 'adaptive_alpha': 0.8, 'noSpeciesBackprop': False, 'phylogeny_loss': 'KLDiv', 'phylogeny_loss_epsilon': 0.03}
Creating dataset...
Loading dataset...
/raid/elhamod/cifar-100-python/
Files already downloaded and verified
CIFAR normalization
Loading dataset...
/raid/elhamod/cifar-100-python/
Files already downloaded and verified
CIFAR normalization
Creating dataset... Done.
train/val =  40000 10000
file /raid/elhamod/CIFAR_HGNN/experiments/CIFAR_phylogeny_8_4_0_KLDiv/trainingIndex.pkl written
file /raid/elhamod/CIFAR_HGNN/experiments/CIFAR_phylogeny_8_4_0_KLDiv/valIndex.pkl written
Creating loaders...
Creating loaders... Done.


{'experimentName': 'CIFAR_phylogeny_8_4_0_KLDiv', 'modelName': 'models/6b63c036ce74bd9c2360a1efc05e196e84807db2fdb81b09fbbdc01f', 'datasetName': 'datasplits/dd10c35154ee995db5de0276a21ca1a15a77964ee53811d98f089e89', 'experimentHash': '843245f82691b9ea95cc530cd93dd7d6b3a46cacb99e0ce433b37eab', 'trialHash': '6b63c036ce74bd9c2360a1efc05e196e84807db2fdb81b09fbbdc01f', 'image_path': 'cifar-100-python', 'suffix': None, 'img_res': 32, 'augmented': False, 'batchSize': 128, 'learning_rate': 5e-05, 'numOfTrials': 5, 'fc_layers': 1, 'modelType': 'BB', 'lambda': 0.01, 'unsupervisedOnTest': False, 'tl_model': 'CIFAR', 'link_layer': 'avgpool', 'adaptive_smoothing': False, 'adaptive_lambda': 0.015, 'adaptive_alpha': 0.8, 'noSpeciesBackprop': False, 'phylogeny_loss': 'KLDiv', 'phylogeny_loss_epsilon': 0.03}


wandb: wandb version 0.10.27 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade




iteration:   0%|          | 0/500 [00:00<?, ?it/s]

...

Early stopping
total number of epochs:  17


iteration:   4%|▎         | 18/500 [21:08<9:26:01, 70.46s/it, min_val_loss=1.21, train=0.992, val=0.812, val_loss=4.14]


loss,0.00778
batch,5633
epoch,17
loss_fine,0.00778
loss_coarse,0
lambda_fine,1
lambda_coarse,0.01
_step,5651
_runtime,1266
_timestamp,1619205426
validation_fine_f1,0.82792


loss,█▇▆▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇█████
loss_fine,█▇▆▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_coarse,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lambda_fine,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lambda_coarse,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇████
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇████
validation_fine_f1,▁▄▅▆▇▇████████████


experiment: 100%|██████████| 1/1 [1:48:14<00:00, 6494.09s/it]